Start Prediction

In [17]:
import warnings
warnings.simplefilter(action='ignore')
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error


In [18]:
def spiltset(Target,filename):
    #train_test_spilt
    df = pd.read_csv(filename)
    df = df.drop(['ID'],axis = 1)
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=0,subset = [Target])
    y = df[Target]
    x = df.drop([Target],axis = 1)
    xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15,random_state =10)
    return x, y, xtrain, xtest, ytrain, ytest

def xgb_best_modle(xtrain, ytrain,X,y):
    #grid search for better model params
    # params = { 'max_depth': [3],
    #        'learning_rate': [0.01, 0.05, 0.1],
    #        'n_estimators': [800,1000],
    #        'colsample_bytree': [0.8],
    #         'lambda':[0],
    #         'alpha':[1]}
    
    params = {
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100,600,1000],
    'max_depth': [1,3],
    'lambda':[0,0.5,1],
    'alpha':[0,0.5,1]
    }

    xgbr = xgb.XGBRegressor(seed = 20)
    clf = GridSearchCV(estimator=xgbr, 
                    param_grid=params,
                    scoring='neg_mean_absolute_percentage_error', 
                    verbose=1,
                    cv=2,
                    )
    clf.fit(xtrain, ytrain)
    print("Best parameters:", clf.best_params_)
    print("Best scores:", clf.best_score_)

    xgb_model = clf.best_estimator_
    scores = cross_val_score(xgbr, X, y, cv=5, scoring='neg_mean_absolute_percentage_error')
    scores =  np.absolute(scores)
    print('MAPE CV Score: %.3f (%.3f)' % (scores.mean(), scores.std()) )
    
    return xgb_model

# def xgb_best_modle(X,y,xtrain, xtest, ytrain, ytest,Target,parmas,flag = False):
#     #fitting in data for the best model and use cv scoreing the model
#     xgbr = xgb.XGBRegressor(seed = 20,max_depth = parmas['max_depth'],learning_rate= parmas['learning_rate'],n_estimators= parmas['n_estimators'],colsample_bytree= parmas['colsample_bytree'])

#     scores = cross_val_score(xgbr, X, y, cv=5, scoring='neg_mean_absolute_percentage_error')
#     scores =  np.absolute(scores)
#     print('MAPE CV Score: %.3f (%.3f)' % (scores.mean(), scores.std()) )

#     xgbr.fit(xtrain, ytrain)
#     ypred = xgbr.predict(xtest)
#     #print("Lowest MAPE(test_set): ", mean_absolute_percentage_error(ytest,ypred))

#     #make plot for test true and predicted value
#     if flag:
#         x_ax = range(len(ytest))
#         plt.plot(x_ax, ytest, label="original")
#         plt.plot(x_ax, ypred, label="predicted")
#         plt.title(Target)
#         plt.legend()
#         plt.show()
#     return xgbr
    
def predict(previous_file,Target,xgbr,newfilename,p_v):
    #perdicting the values
    previous_df = pd.read_csv(previous_file)
    orginal_df = pd.read_csv('./data/REST_data_1.csv')

    mydf = orginal_df[pd.isnull(orginal_df[Target])]
    true_x_for_prediction = mydf.drop(columns=['ID',Target],inplace= False)
    ypred = xgbr.predict(true_x_for_prediction)

    ypred = [ '%.2f' % elem for elem in ypred ]

    mydf[Target] = ypred
    mydf[Target] = mydf[Target].apply(lambda x: f"{x}"+p_v)

    previous_df.set_index(['ID'], inplace=True)
    previous_df.update(mydf.set_index(['ID']))
    previous_df.reset_index( inplace=True)
    previous_df.to_csv(newfilename,index = False)

In [19]:
#Choose good models and mark as first round of perdiction, ready to make second round

#first round
Target = "REST: Staff size MOD" 
print("Prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1.csv','(p1)')
print("-----Prediction END-------")

Targetlist = [ "REST: No Wtr MOD","REST: No Ktch Wkr MOD","REST: No tables MOD","REST: Monthly Expenses MOD"]
for Target in Targetlist:
    print("Prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1.csv','(p1)')
    print("-----Prediction END-------")



Prediction start for: REST: Staff size MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.4712055346919227
MAPE CV Score: 0.346 (0.154)
-----Prediction END-------
Prediction start for: REST: No Wtr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0.5, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.31172121821608
MAPE CV Score: 0.222 (0.089)
-----Prediction END-------
Prediction start for: REST: No Ktch Wkr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 600}
Best scores: -0.233649934857513
MAPE CV Score: 0.297 (0.193)
-----Prediction END-------
Prediction start for: REST: No tables MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best 

In [22]:
#first round

Target = "REST: Staff size MOD" 
print("First round prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1_without_label.csv','')
print("-----Prediction END-------")

Targetlist = [ "REST: No Wtr MOD","REST: No Ktch Wkr MOD","REST: No tables MOD","REST: Monthly Expenses MOD"]
for Target in Targetlist:
    print("First round prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v1_without_label.csv',Target, xgbr,'./data/REST_data_1_predicted_v1_without_label.csv','')
    print("-----Prediction END-------")


#second round
Target = "REST: Monthly Expenses MOD"
print("Second round prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v1_without_label.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1_predicted_v1.csv',Target, xgbr,'./data/REST_data_1_predicted_v2.csv','(p2)')
print("-----Prediction END-------")

Targetlist = ["REST: Monthly Sales MOD","REST: Dividends MOD","REST: Monthly rent MOD","REST: Total Capital MOD","REST: Total Partners MOD","REST: No Act Part MOD","REST:  No Pass Part MOD"]
for Target in Targetlist:
    print("Second round prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v1_without_label.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v2.csv',Target, xgbr,'./data/REST_data_1_predicted_v2.csv','(p2)')

First round prediction start for: REST: Staff size MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.4712055346919227
MAPE CV Score: 0.346 (0.154)
-----Prediction END-------
First round prediction start for: REST: No Wtr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0.5, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.31172121821608
MAPE CV Score: 0.222 (0.089)
-----Prediction END-------
First round prediction start for: REST: No Ktch Wkr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 600}
Best scores: -0.233649934857513
MAPE CV Score: 0.297 (0.193)
-----Prediction END-------
First round prediction start for: REST: No tables MOD
Fitting 2 folds for 

In [23]:
#first round
Target = "REST: Staff size MOD" 
print("First round prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1_without_label.csv','')
predict('./data/REST_data_1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1.csv','(p1)')
print("-----Prediction END-------")

Targetlist = [ "REST: No Wtr MOD","REST: No Ktch Wkr MOD","REST: No tables MOD","REST: Monthly Expenses MOD"]
for Target in Targetlist:
    print("First round prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_Cleaned.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v1_without_label.csv',Target, xgbr,'./data/REST_data_1_predicted_v1_without_label.csv','')
    predict('./data/REST_data_1_predicted_v1.csv',Target, xgbr,'./data/REST_data_1_predicted_v1.csv','(p1)')
    print("-----Prediction END-------")


#second round
Target = "REST: Monthly Expenses MOD"
print("Second round prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v1_without_label.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1_predicted_v1_without_label.csv',Target, xgbr,'./data/REST_data_1_predicted_v2_without_label.csv','')
predict('./data/REST_data_1_predicted_v1.csv',Target, xgbr,'./data/REST_data_1_predicted_v2.csv','(p2)')
print("-----Prediction END-------")

Targetlist = ["REST: Total Partners MOD","REST: No Act Part MOD","REST:  No Pass Part MOD"]
for Target in Targetlist:
    print("Second round prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v1_without_label.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v2_without_label.csv',Target, xgbr,'./data/REST_data_1_predicted_v2_without_label.csv','')
    predict('./data/REST_data_1_predicted_v2.csv',Target, xgbr,'./data/REST_data_1_predicted_v2.csv','(p2)')
    print("-----Prediction END-------")

#third round
Target = "REST: Monthly Sales MOD"
print("Third round prediction start for:",Target)
x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v2_without_label.csv')
xgbr = xgb_best_modle(xtrain,ytrain, x,y)
predict('./data/REST_data_1_predicted_v2.csv',Target, xgbr,'./data/REST_data_1_predicted_v3.csv','(p3)')

print("-----Prediction END-------")

Targetlist = ["REST: Dividends MOD","REST: Monthly rent MOD","REST: Total Capital MOD"]
for Target in Targetlist:
    print("Second round prediction start for:",Target)
    x,y,xtrain, xtest, ytrain, ytest = spiltset(Target,'./data/REST_data_1_predicted_v2_without_label.csv')
    xgbr = xgb_best_modle(xtrain,ytrain, x,y)
    predict('./data/REST_data_1_predicted_v3.csv',Target, xgbr,'./data/REST_data_1_predicted_v3.csv','(p3)')
    print("-----Prediction END-------")


First round prediction start for: REST: Staff size MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.4712055346919227
MAPE CV Score: 0.346 (0.154)
-----Prediction END-------
First round prediction start for: REST: No Wtr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0.5, 'lambda': 1, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best scores: -0.31172121821608
MAPE CV Score: 0.222 (0.089)
-----Prediction END-------
First round prediction start for: REST: No Ktch Wkr MOD
Fitting 2 folds for each of 162 candidates, totalling 324 fits
Best parameters: {'alpha': 0, 'lambda': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 600}
Best scores: -0.233649934857513
MAPE CV Score: 0.297 (0.193)
-----Prediction END-------
First round prediction start for: REST: No tables MOD
Fitting 2 folds for 

## MAPE test set
"REST: Monthly Sales MOD" 0.6838249290054842 0.38120734969774417

 "REST: Monthly rent MOD" 0.4421961655441294

 "REST: Monthly Expenses MOD" 1.355231134400035 1.3793507225600399

 "REST: Dividends MOD" 0.8470890341158267 0.707422894583808

 "REST: Staff size MOD" 0.17751424270084445

 "REST: No Wtr MOD" 0.14249820715596534

 "REST: No Ktch Wkr MOD" 0.6896845638751984 0.7351611022911373

 "REST: No tables MOD" 0.32794427786449615

 "REST: Total Capital MOD" 0.3600939190193398

 "REST: Total Partners MOD" 0.26090539669634416

 "REST: No Act Part MOD" 0.18370731953988995
 
 "REST:  No Pass Part MOD" 218725280819882.8


## MAPE CV
"REST: Monthly Sales MOD" 0.79 0.796

 "REST: Monthly rent MOD" 0.538 0.22

 "REST: Monthly Expenses MOD" 0.44 0.44

 "REST: Dividends MOD" 0.872 0.914 

 "REST: Staff size MOD" 0.352

 "REST: No Wtr MOD" 0.239

 "REST: No Ktch Wkr MOD" 0.303

 "REST: No tables MOD" 0.295

 "REST: Total Capital MOD" 0.649 0.210 

 "REST: Total Partners MOD" 0.492 0.126
 
 "REST: No Act Part MOD" 0.484 0.145
 
 "REST:  No Pass Part MOD" error (0 value exist)
